### Load Data Input 

In [1]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
#print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed
print(bucket)
prefix = 'NLP_AWS/NLP_AWS' #Replace with the prefix under which you want to store the data if needed

sagemaker-us-east-1-023375022819


In [3]:
!wget https://s3.amazonaws.com/fast-ai-nlp/amazon_review_polarity_csv.tgz

--2019-08-28 02:25:12--  https://s3.amazonaws.com/fast-ai-nlp/amazon_review_polarity_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.205.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.205.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 688339454 (656M) [application/x-tar]
Saving to: ‘amazon_review_polarity_csv.tgz.2’

amazon_review_polar 100%[===================>] 656.45M  52.5MB/s    in 14s     

2019-08-28 02:25:26 (46.9 MB/s) - ‘amazon_review_polarity_csv.tgz.2’ saved [688339454/688339454]



In [4]:
!tar -xzvf amazon_review_polarity_csv.tgz

amazon_review_polarity_csv/
amazon_review_polarity_csv/train.csv
amazon_review_polarity_csv/readme.txt
amazon_review_polarity_csv/test.csv


In [2]:
!head amazon_review_polarity_csv/train.csv -n 3

"2","Stuning even for the non-gamer","This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^"
"2","The best soundtrack ever to anything.","I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."
"2","Amazing!","This soundtrack is my favorite mu

#### Then upload file from sagemaker folder to S3 

In [10]:
%%time

train_channel = prefix + '/train'
test_channel = prefix + '/test'

sess.upload_data(path='amazon_review_polarity_csv/train.csv', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='amazon_review_polarity_csv/test.csv', bucket=bucket, key_prefix=test_channel)

CPU times: user 12.7 s, sys: 6.89 s, total: 19.6 s
Wall time: 18.9 s


### Read datafile as CSV

In [2]:
import pandas as pd

df = pd.read_csv("s3://sagemaker-us-east-1-023375022819/NLP_AWS/NLP_AWS/train/train.csv", names=["Label", "Title", "Review"])

In [3]:
neg_df = df[df['Label'] == 1]

In [4]:
neg_df.head()

,Label,Title,Review
6,1,Buyer beware,"This is a self-published book, and if you want..."
10,1,The Worst!,A complete waste of time. Typographical errors...
13,1,Oh please,I guess you have to be a romance novel lover f...
14,1,Awful beyond belief!,I feel I have to write to keep others from was...
15,1,Don't try to fool us with fake reviews.,It's glaringly obvious that all of the glowing...


In [5]:
neg_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1800000 entries, 6 to 3599998
Data columns (total 3 columns):
Label     int64
Title     object
Review    object
dtypes: int64(1), object(2)
memory usage: 54.9+ MB


In [15]:
extract_df = neg_df.iloc[:,0:10000]['Review']

In [16]:
extract_df.head(10)

6     This is a self-published book, and if you want...
10    A complete waste of time. Typographical errors...
13    I guess you have to be a romance novel lover f...
14    I feel I have to write to keep others from was...
15    It's glaringly obvious that all of the glowing...
19    sizes are much smaller than what is recomended...
20    This model may be ok for sedentary types, but ...
22    Rather than scratches and insect droppings, th...
25    I have had the charger for more than two years...
26    I bought one of these chargers..the instructio...
Name: Review, dtype: object

In [19]:
extract_df.count()

1800000

In [17]:
extract_df.to_csv('extract_df.csv', index=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


In [ ]:
comprehend_channel = prefix + '/comprehend'

sess.upload_data(path='extract_df.csv', bucket=bucket, key_prefix=comprehend_channel)

In [16]:
!wget https://s3.console.aws.amazon.com/s3/object/sagemaker-us-east-1-023375022819/NLP_AWS/NLP_AWS/023375022819-TOPICS-f0b9b40086d3cc882f48800be187f3fb/output/output.tar.gz

--2019-08-28 15:31:17--  https://s3.console.aws.amazon.com/s3/object/sagemaker-us-east-1-023375022819/NLP_AWS/NLP_AWS/023375022819-TOPICS-f0b9b40086d3cc882f48800be187f3fb/output/output.tar.gz
Resolving s3.console.aws.amazon.com (s3.console.aws.amazon.com)... 54.239.31.91
Connecting to s3.console.aws.amazon.com (s3.console.aws.amazon.com)|54.239.31.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 989 [text/html]
Saving to: ‘output.tar.gz’

output.tar.gz       100%[===================>]     989  --.-KB/s    in 0s      

2019-08-28 15:31:17 (135 MB/s) - ‘output.tar.gz’ saved [989/989]



In [21]:
!aws s3 cp s3://sagemaker-us-east-1-023375022819/NLP_AWS/NLP_AWS/023375022819-TOPICS-f0b9b40086d3cc882f48800be187f3fb/output/output.tar.gz .

download: s3://sagemaker-us-east-1-023375022819/NLP_AWS/NLP_AWS/023375022819-TOPICS-f0b9b40086d3cc882f48800be187f3fb/output/output.tar.gz to ./output.tar.gz


In [22]:
!tar -xvzf output.tar.gz   

topic-terms.csv
doc-topics.csv


In [23]:
!cat topic-terms.csv

topic,term,weight
000,item,0.01320014
000,price,0.009271662
000,cheap,0.008077917
000,product,0.0145492125
000,return,0.00996442
000,buy,0.022574656
000,worth,0.0070977435
000,pay,0.005717317
000,wear,0.0050701024
000,shirt,0.0037684275
001,movie,0.0891871
001,watch,0.026982259
001,make,0.020209568
001,wrong,0.016609019
001,time,0.018557215
001,funny,0.0075101038
001,waste,0.011658727
001,plot,0.008867379
001,bore,0.011109356
001,minute,0.006217682
002,book,0.082376644
002,story,0.030742362
002,read,0.026288286
002,character,0.01778436
002,write,0.018424777
002,find,0.017625522
002,interest,0.014726332
002,plot,0.012445922
002,good,0.019640913
002,bore,0.013027136
003,film,0.04789476
003,bad,0.023904858
003,movie,0.023667585
003,act,0.011379912
003,star,0.010218274
003,script,0.008100315
003,make,0.013481576
003,watch,0.011116439
003,keanu,0.0064930655
003,performance,0.0065185176
004,work,0.061424877
004,product,0.039028663
004,great,0.015444886
004,week,0.0121942
004,stop,0.010744139

### Deploy Neutral Topic Modeling

In [22]:
short_df = extract_df.head(10000)

In [23]:
short_df.count()

10000

In [24]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import re
token_pattern = re.compile(r"(?u)\b\w\w+\b")
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if len(t) >= 2 and re.match("[a-z].*",t) 
                and re.match(token_pattern, t)]

OSError: [Errno 12] Cannot allocate memory

In [9]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
import time
import numpy as np

In [11]:
!conda install -y scipy

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [25]:
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
vocab_size = 2000
print('Tokenizing and counting, this may take a few minutes...')
start_time = time.time()

Tokenizing and counting, this may take a few minutes...


In [28]:
vectorizer = CountVectorizer(input='content', analyzer='word', stop_words='english',
                             tokenizer=LemmaTokenizer(), max_features=vocab_size, max_df=0.95, min_df=2)
vectors = vectorizer.fit_transform(short_df)
vocab_list = vectorizer.get_feature_names()
print('vocab size:', len(vocab_list))

# random shuffle
idx = np.arange(vectors.shape[0])
np.random.shuffle(idx)
vectors = vectors[idx]

print('Done. Time elapsed: {:.2f}s'.format(time.time() - start_time))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


vocab size: 2000
Done. Time elapsed: 50.94s
